In [ ]:
%matplotlib inline
#%matplotlib notebook

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import cv2
import numpy as np
import random

from ipywidgets import FloatSlider, IntSlider
from ipywidgets import interactive
from utils_noise import colorize, normalize, normalize_var
from utils_noise import plot_colored, plot_spectral
from utils_noise import gaborK, gaborN_rand, gaborN_uni

# Gabor Kernel

A Gabor kernel is the convolution of a Gaussian and Harmonic kernel. This results in symmetric gaussian lobes in its power spectrum.
1. **sigma** - Variance of the Gaussian.
2. **theta** - Orientation or angle of the kernel.
3. **lambd** - Wavelength of the sinusoidal factor in the Harmonic kernel. In the spectral domain, this changes the distance between the lobes.

In [ ]:
ksize = 199 # fixed kernel size

# Interactive slider
def PlotGaborK(sigma, theta, lambd, xy_ratio, sides):
    img = gaborK(ksize = ksize, sigma = sigma, theta = theta, lambd = lambd, xy_ratio = xy_ratio, sides = sides)
    plot_spectral(img, title = 'Gabor Kernel')
    
# Parameter sliders
s_sigma = FloatSlider(min = 2, max = 20, value = 8, step = 0.5, continuous_update = False, description = 'Kernel Var:')
s_theta = FloatSlider(min = 0, max = np.pi, value = np.pi / 4, step = np.pi / 20, continuous_update = False, description = 'Orientation:')
s_lambd = FloatSlider(min = 0.25, max = 20, value = 10, step = 0.25, continuous_update = False, description = 'Bandwidth:')
s_xyratio = FloatSlider(min = 0.1, max = 10, value = 1, step = 0.1, continuous_update = False, description = 'XY Ratio:')
s_sides = IntSlider(min = 1, max = 12, value = 1, step = 1, continuous_update = False, description = 'Sides:')

interactive(PlotGaborK,
                 sigma = s_sigma,
                 theta = s_theta,
                 lambd = s_lambd,
                 xy_ratio = s_xyratio,
                 sides = s_sides)

# Gabor Noise

A Gabor noise is the convolution of a sparse white noise and Gabor kernel, making it a type of "Sparse Convolution Noise". Informally, this can be thought of as *splatting* an image with Gabor kernels. Sparse white noise typically has a flat power spectrum, hence the spectral properties of the Gabor noise is equivalent to its Gabor kernel.

### Random Spread

The kernels here share the same properties, but are randomly distributed across the image.
1. **grid** - Size of the grid the image is divided into.
2. **num_kern** - Number of kernels added to each grid.
3. **kszie** - Size of each kernel. When kernel size is reduced to one, the resulting image will just be the sparse white noise.

In [ ]:
size = 224 # fixed image size
grid = 14 # fixed grid size
# num_kern is the number of kernels per grid

def PlotGaborAni(num_kern, ksize, sigma, theta, lambd, sides):
    img = gaborN_rand(size = size, grid = grid, num_kern = num_kern, ksize = ksize, sigma = sigma, theta = theta, lambd = lambd, sides = sides)
    img = normalize_var(img)
    plot_colored(img, title = 'Gabor Noise - Random')
    
# Parameter sliders
s_num_kern = IntSlider(min = 1, max = 200, value = 23, step = 1, continuous_update = False, description = 'No. Kernels:')
s_ksize = IntSlider(min = 1, max = 40, value = 23, step = 1, continuous_update = False, description = 'Kernel Size:')
s_sigma = FloatSlider(min = 0, max = 20, value = 8, step = 0.25, continuous_update = False, description = 'Kernel Var:')
s_theta = FloatSlider(min = 0, max = np.pi, value = np.pi / 4, step = np.pi / 24, continuous_update = False, description = 'Orientation:')
s_lambd = FloatSlider(min = 1, max = 20, value = 8, step = 0.25, continuous_update = False, description = 'Bandwidth:')
s_sides = IntSlider(min = 1, max = 12, value = 1, step = 1, continuous_update = False, description = 'Sides:')

interactive(PlotGaborAni,
                 num_kern = s_num_kern,
                 ksize = s_ksize,
                 sigma = s_sigma,
                 theta = s_theta,
                 lambd = s_lambd,
                 sides = s_sides)

## Uniform Spread, Random Orientations

The kernels here share the same properties except orientation. They are uniformly istributed across the image for more predictable control. Most of the parameters are fixed with the exception of **lambd** which controls the thickness of the kernels. The intention is to have individual orientations for each kernel uniformly spaced along the grid.

In [ ]:
'''
size              size of image
grid              best if odd and a factor of size
ksize            slightly larger than grid_size so that kernels merge or look more cohesive
sigma           large enough so that adjacent kernels merge
'''
size = 299
grid = 13
ksize = grid + 4
sigma = grid // 3
xy_ratio = 1

# Random orientations for kernels
no_filters = (299 // 13)**2
np.random.seed(0)
thetas = np.random.uniform(low = 0, high = np.pi, size = no_filters)

# Interactive slider
def PlotGaborAni(lambd):
    img = gaborN_uni(size = size, grid = grid, ksize = ksize, sigma = sigma, lambd = lambd, thetas = thetas, xy_ratio = xy_ratio)
    img = normalize_var(img)
    plot_colored(img, title = 'Gabor Noise - Uniform')
    
s_lambd = FloatSlider(min = 0.25, max = grid, value = 7, step = 0.25, continuous_update = False, description = 'Bandwidth:')
interactive(PlotGaborAni, lambd = s_lambd)